In [ ]:
import tkinter as tk
from tkinter import messagebox, filedialog
from datetime import datetime
import threading
import time

class Task:
    def __init__(self, name, due_date, priority):
        self.name = name
        self.due_date = due_date
        self.priority = priority
        self.completed = False

class TimeManager:
    def __init__(self):
        self.tasks = []

    def add_task(self, name, due_date, priority):
        task = Task(name, due_date, priority)
        self.tasks.append(task)

    def complete_task(self, name):
        for task in self.tasks:
            if task.name == name:
                task.completed = True
                return True
        return False

    def finish_task(self, name):
        for task in self.tasks:
            task.completed = True
            return True
        return False

    def edit_task(self, old_name, new_name, due_date, priority):
        for task in self.tasks:
            if task.name == old_name:
                task.name = new_name
                task.due_date = due_date
                task.priority = priority
                return True
        return False

    def delete_task(self, name):
        self.tasks = [task for task in self.tasks if task.name != name]

    def get_tasks(self):
        return self.tasks

    def save_tasks(self, filename):
        with open(filename, 'w') as f:
            for task in self.tasks:
                f.write(f"{task.name},{task.due_date},{task.priority},{task.completed}\n")

    def load_tasks(self, filename):
        self.tasks = []
        with open(filename, 'r') as f:
            for line in f:
                name, due_date_str, priority, completed_str = line.strip().split(',')
                due_date = datetime.strptime(due_date_str, '%Y-%m-%d %H:%M')
                priority = int(priority)
                completed = completed_str == 'True'
                task = Task(name, due_date, priority)
                task.completed = completed
                self.tasks.append(task)

class App:
    def __init__(self, root):
        self.manager = TimeManager()
        self.root = root
        self.root.title("Time Management Assistant")

        # Task Entry
        self.task_name_var = tk.StringVar()
        self.due_date_var = tk.StringVar()
        self.priority_var = tk.IntVar()

        tk.Label(root, text="Task Name:").grid(row=0, column=0)
        tk.Entry(root, textvariable=self.task_name_var).grid(row=0, column=1)

        tk.Label(root, text="Due Date (YYYY-MM-DD HH:MM):").grid(row=1, column=0)
        tk.Entry(root, textvariable=self.due_date_var).grid(row=1, column=1)

        tk.Label(root, text="Priority (1-High, 2-Medium, 3-Low):").grid(row=2, column=0)
        tk.Entry(root, textvariable=self.priority_var).grid(row=2, column=1)

        tk.Button(root, text="Add Task", command=self.add_task).grid(row=3, column=0, columnspan=2)
        tk.Button(root, text="Edit Task", command=self.edit_task).grid(row=3, column=1)
        tk.Button(root, text="Delete Task", command=self.delete_task).grid(row=3, column=2)
        tk.Button(root, text="Finish Task", command=self.finish_task).grid(row=3, column=3)

        self.task_listbox = tk.Listbox(root, width=50)
        self.task_listbox.grid(row=4, column=0, columnspan=4)

        tk.Button(root, text="Complete Task", command=self.delete_task).grid(row=5, column=0, columnspan=4)
        tk.Button(root, text="Save Tasks", command=self.save_tasks).grid(row=6, column=0)
        tk.Button (root, text="Load Tasks", command=self.load_tasks).grid(row=6, column=1)

        self.reminder_thread = threading.Thread(target=self.remind_tasks)
        self.reminder_thread.daemon = True
        self.reminder_thread.start()

    def add_task(self):
        name = self.task_name_var.get()
        due_date_str = self.due_date_var.get()
        priority = self.priority_var.get()

        try:
            due_date = datetime.strptime(due_date_str, '%Y-%m-%d %H:%M')
            self.manager.add_task(name, due_date, priority)
            self.update_task_list()
            self.task_name_var.set("")
            self.due_date_var.set("")
            self.priority_var.set(0)
        except ValueError:
            messagebox.showerror("Invalid date", "Please enter a valid date in the format YYYY-MM-DD HH:MM")

    def edit_task(self):
        selected_task = self.task_listbox.curselection()
        if selected_task:
            old_name = self.task_listbox.get(selected_task)
            new_name = self.task_name_var.get()
            due_date_str = self.due_date_var.get()
            priority = self.priority_var.get()

            try:
                due_date = datetime.strptime(due_date_str, '%Y-%m-%d %H:%M')
                if self.manager.edit_task(old_name, new_name, due_date, priority):
                    self.update_task_list()
                    messagebox.showinfo("Task Edited", f"Task '{old_name}' has been edited.")
                else:
                    messagebox.showerror("Error", "Task not found.")
            except ValueError:
                messagebox.showerror("Invalid date", "Please enter a valid date in the format YYYY-MM-DD HH:MM")
        else:
            messagebox.showwarning("Select Task", "Please select a task to edit.")

    def delete_task(self):
        selected_task = self.task_listbox.curselection()
        if selected_task:
            task_name = self.task_listbox.get(selected_task)
            self.manager.delete_task(task_name)
            self.update_task_list()
            messagebox.showinfo("Task Deleted", f"Task '{task_name}' has been deleted.")
        else:
            messagebox.showwarning("Select Task", "Please select a task to delete.")

    def finish_task(self):
        selected_task = self.task_listbox.curselection()
        if selected_task:
            task_name = self.task_listbox.get(selected_task)
            if self.manager.finish_task(task_name):
                self.update_task_list()
                messagebox.showinfo("Task Finished", f"Task '{task_name}' has been marked as finished.")
            else:
                messagebox.showerror("Error", "Task not found.")
        else:
            messagebox.showwarning("Select Task", "Please select a task to finish.")

    def save_tasks(self):
        filename = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
        if filename:
            self.manager.save_tasks(filename)
            messagebox.showinfo("Tasks Saved", "Tasks have been saved successfully.")

    def load_tasks(self):
        filename = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])
        if filename:
            self.manager.load_tasks(filename)
            self.update_task_list()
            messagebox.showinfo("Tasks Loaded", "Tasks have been loaded successfully.")

    def complete_task(self):
        selected_task = self.task_listbox.curselection()
        if selected_task:
            task_name = self.task_listbox.get(selected_task)
            if self.manager.complete_task(task_name):
                self.update_task_list()
                messagebox.showinfo("Task Completed", f"Task '{task_name}' has been completed.")
            else:
                messagebox.showerror("Error", "Task not found.")
        else:
            messagebox.showwarning("Select Task", "Please select a task to complete.")

    def update_task_list(self):
        self.task_listbox.delete(0, tk.END)
        for task in self.manager.get_tasks():
            status = "✔️" if task.completed else "❌"
            self.task_listbox.insert(tk.END, f"{status} {task.name} - Due: {task.due_date} - Priority: {task.priority}")

    def remind_tasks(self):
        while True:
            now = datetime.now()
            for task in self.manager.get_tasks():
                if not task.completed and task.due_date <= now:
                    messagebox.showinfo("Reminder", f"Task '{task.name}' is due!")
            time.sleep(60)  # Check every minute

if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()